In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys
sys.path.append('../')

In [98]:
from src.preprocessor.preprocessing import FileIO, Utilities
from src.llm.llm_interface import LLM
from tqdm import tqdm
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [90]:
data_path = '../data/guests.json'
guests = FileIO().load_json(data_path)

In [37]:
guest_extraction_system = '''
Your primary role is to extract information from any given text.  Specifically you are an expert at extracting guest names from podcast shows
'''

In [62]:
assistant = '''
You will be given the title of a podcast from the Huberman Labs podcast.  Usually the guest name will be found within the title.  If the guest name is present, return the guest name.  Do not provide any other information or explanation, simply return the extracted guests name. 
If there is no guest name in the title, or if the title starts with the word AMA (which stands for Ask Me Anything), then return the name "Andrew Huberman".
Again if no guest is found in the title, then return the name "Andrew Huberman"
```
{title}
```
'''

In [63]:
llm = LLM()

In [64]:
async def get_guest(title: str):
    guest = await llm.achat_completion(guest_extraction_system, assistant.format(title=title), max_tokens=25, raw_response=True)
    return guest

In [65]:
async def guest_tasks(titles: list[str]):
    tasks = await asyncio.gather(*[get_guest(title) for title in titles])
    return tasks

In [66]:
%%time
output = asyncio.run(guest_tasks(titles))

CPU times: user 8.38 s, sys: 369 ms, total: 8.75 s
Wall time: 10.1 s


In [67]:
len(output)

189

In [69]:
guests = [response.choices[0].message.content for response in output]

In [74]:
import json
with open('../data/guests.json', 'w') as f:
    f.write(json.dumps(guests, indent=4))

### Combine Metadata

In [100]:
raw_data = FileIO().load_json('../data/hubermanlabs.json')
utils = Utilities()

In [101]:
data = utils.convert_raw_data(raw_data)

In [103]:
for i, d in enumerate(data):
    d['guest'] = guests[i]

In [110]:
with open('../data/huberman_labs.json', 'w') as f:
    f.write(json.dumps(data, indent=2))